In [12]:
from pyspark.sql import SparkSession

# Update the path to the downloaded PostgreSQL JDBC jar
jdbc_driver_path = r"D:\I4-AMS-S2\Big_Data\optimize_marketing\Backend\postgresql-42.7.7.jar"

spark = SparkSession.builder \
    .appName("PostgresConnection") \
    .config("spark.jars", jdbc_driver_path) \
    .getOrCreate()

In [13]:
csv_file_path = "./../data/Coffee_Shop_Sales.csv"

# If your CSV has headers
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(csv_file_path)

print(df.head(5))

num_columns = len(df.columns)
print("Number of columns:", num_columns)

num_rows = df.count()
print("Number of rows:", num_rows)

[Row(transaction_id=1, transaction_date='1/1/2023', transaction_time=datetime.datetime(2025, 6, 15, 7, 6, 11), transaction_qty=2, store_id=5, store_location='Lower Manhattan', product_id=32, unit_price=3.0, product_category='Coffee', product_type='Gourmet brewed coffee', product_detail='Ethiopia Rg'), Row(transaction_id=2, transaction_date='1/1/2023', transaction_time=datetime.datetime(2025, 6, 15, 7, 8, 56), transaction_qty=2, store_id=5, store_location='Lower Manhattan', product_id=57, unit_price=3.1, product_category='Tea', product_type='Brewed Chai tea', product_detail='Spicy Eye Opener Chai Lg'), Row(transaction_id=3, transaction_date='1/1/2023', transaction_time=datetime.datetime(2025, 6, 15, 7, 14, 4), transaction_qty=2, store_id=5, store_location='Lower Manhattan', product_id=59, unit_price=4.5, product_category='Drinking Chocolate', product_type='Hot chocolate', product_detail='Dark chocolate Lg'), Row(transaction_id=4, transaction_date='1/1/2023', transaction_time=datetime.da

In [14]:
jdbc_url = "jdbc:postgresql://localhost:5432/coffee_shop"
connection_properties = {
    "user": "jame",
    "password": "admin",
    "driver": "org.postgresql.Driver"
}

In [15]:
# Write DataFrame to PostgreSQL (creates the table if it doesn't exist)
df.write.jdbc(
    url=jdbc_url,
    table="coffee_shop_sales",
    mode="overwrite",
    properties=connection_properties
)

# df.orderBy("transaction_id").write.jdbc(
#     url=jdbc_url,
#     table="coffee_shop_sales",
#     mode="overwrite",
#     properties=connection_properties
# )

In [16]:
# Read the data from PostgreSQL
df_from_db = spark.read.jdbc(
    url=jdbc_url,
    table="coffee_shop_sales",  # or "coffee_shop_sales"
    properties=connection_properties
)
df_from_db.show()

+--------------+----------------+-------------------+---------------+--------+---------------+----------+----------+------------------+--------------------+--------------------+
|transaction_id|transaction_date|   transaction_time|transaction_qty|store_id| store_location|product_id|unit_price|  product_category|        product_type|      product_detail|
+--------------+----------------+-------------------+---------------+--------+---------------+----------+----------+------------------+--------------------+--------------------+
|        138132|       6/21/2023|2025-06-15 08:30:42|              2|       8| Hell's Kitchen|        31|       2.2|            Coffee|Gourmet brewed co...|         Ethiopia Sm|
|        138133|       6/21/2023|2025-06-15 08:30:47|              1|       8| Hell's Kitchen|        59|       4.5|Drinking Chocolate|       Hot chocolate|   Dark chocolate Lg|
|        138134|       6/21/2023|2025-06-15 08:32:23|              2|       8| Hell's Kitchen|        56|     

In [17]:
df_from_db = df_from_db.orderBy("transaction_id")

num_columns = len(df_from_db.columns)
print("Number of columns:", num_columns)

num_rows = df_from_db.count()
print("Number of rows:", num_rows)

df_from_db.show(5)

Number of columns: 11
Number of rows: 149116
+--------------+----------------+-------------------+---------------+--------+---------------+----------+----------+------------------+--------------------+--------------------+
|transaction_id|transaction_date|   transaction_time|transaction_qty|store_id| store_location|product_id|unit_price|  product_category|        product_type|      product_detail|
+--------------+----------------+-------------------+---------------+--------+---------------+----------+----------+------------------+--------------------+--------------------+
|             1|        1/1/2023|2025-06-15 07:06:11|              2|       5|Lower Manhattan|        32|       3.0|            Coffee|Gourmet brewed co...|         Ethiopia Rg|
|             2|        1/1/2023|2025-06-15 07:08:56|              2|       5|Lower Manhattan|        57|       3.1|               Tea|     Brewed Chai tea|Spicy Eye Opener ...|
|             3|        1/1/2023|2025-06-15 07:14:04|            

In [18]:
import pandas as pd

pdf = df_from_db.toPandas()

pdf.to_csv('./../Data/coffee_shop_export.csv', index=False)
